***Image Similarity***

In [95]:
import os
import cv2
import numpy as np
from scipy.spatial.distance import cosine

In [96]:
def preprocess_image(image_path, desired_width=256, desired_height=256):
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image file '{image_path}' not found.")
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Failed to load image from '{image_path}'.")
    if image.size == 0:       # Resizing the image 
        raise ValueError("Loaded image has empty size.")
    image = cv2.resize(image, (desired_width, desired_height))
    # Converting the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray_image

def compute_features(image):
    hog = cv2.HOGDescriptor()
    features = hog.compute(image)
    
    return features


In [97]:
def load_indexed_features(features_output_folder):
    indexed_features = {}
    for filename in os.listdir(features_output_folder):
        if filename.endswith('.npz'):
            features_output_path = os.path.join(features_output_folder, filename)
            features_data = np.load(features_output_path)
            indexed_features[filename[:-4]] = features_data['features']
    
    return indexed_features


In [98]:
def search_image(query_name, indexed_features):
    if query_name not in indexed_features:
        print(f"No images found for '{query_name}'.")
        return []
    query_features = indexed_features[query_name]
    similarity_scores = {}
    for image_name, features in indexed_features.items():
        # Computing cosine distance between features
        similarity_scores[image_name] = cosine(query_features.flatten(), features.flatten())
    
    # Sorting the images based on similarity scores
    sorted_images = sorted(similarity_scores.items(), key=lambda x: x[1])
    
    # Returning top 5 matching image names
    top_matches = [image_name for image_name, _ in sorted_images[:5]]
    return top_matches

folder_path = r'C:\Users\Dell\Pictures\PAI_Project\data\images'
features_output_folder = r'C:\Users\Dell\Pictures\PAI_Project\data\features'

indexed_features = load_indexed_features(features_output_folder)

query_name = input("Enter the name of the image you want to search for: ")

# Searching
top_matches = search_image(query_name, indexed_features)
if top_matches:
    print("Top 5 matching images:")
    for match in top_matches:
        print(match)
else:
    print("No matching images found.")


Enter the name of the image you want to search for: panda
Top 5 matching images:
panda
wolf2
persona
dog
tiger
